In [9]:
!pip install -q groq pdfplumber fpdf pandas openpyxl

import os
import pandas as pd
from groq import Groq
from fpdf import FPDF
from getpass import getpass
import pdfplumber

# === CONFIGURAÇÕES ===
ARQUIVO = "RetailInsights.xlsx"
SAIDA = "relatorio_vendas.pdf"

# === SOLICITAR CHAVE DE API ===
api_key = getpass("🔑 Digite sua chave da API da Groq (os caracteres não serão exibidos): ")

# === LER DADOS COM RESUMO ===
def ler_dados(arquivo):
    if arquivo.endswith(".xlsx"):
        df = pd.read_excel(arquivo)
        resumo = df.describe(include='all').to_string()
        amostra = df.head(10).to_string(index=False)
        return f"Resumo estatístico dos dados:\n{resumo}\n\nAmostra (10 primeiras linhas):\n{amostra}"
    elif arquivo.endswith(".pdf"):
        texto = ""
        with pdfplumber.open(arquivo) as pdf:
            for pagina in pdf.pages[:2]:
                texto += pagina.extract_text() + "\n"
        return texto
    else:
        raise ValueError("Formato não suportado: use Excel ou PDF")

dados = ler_dados(ARQUIVO)

# === CONECTAR À GROQ ===
client = Groq(api_key=api_key)

prompt = f"""
Você é um analista de vendas. Gere um relatório executivo a partir do resumo abaixo.
Inclua:
- Resumo geral de vendas
- Produtos/serviços com melhor desempenho
- Análise de tendências
- Recomendações estratégicas

Resumo dos dados:
{dados}
"""

response = client.chat.completions.create(
    model="llama-3.1-8b-instant",  # ✅ modelo atualizado
    messages=[{"role": "user", "content": prompt}],
)

relatorio_texto = response.choices[0].message.content

# === GERAR PDF ===
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
for linha in relatorio_texto.split("\n"):
    pdf.multi_cell(0, 10, linha)
pdf.output(SAIDA)

print(f"\n✅ Relatório gerado com sucesso: {SAIDA}")


🔑 Digite sua chave da API da Groq (os caracteres não serão exibidos): ··········

✅ Relatório gerado com sucesso: relatorio_vendas.pdf
